In [1]:
import json
import outlines
from transformers import AutoTokenizer
import torch
from textwrap import dedent

In [2]:
model_name = "microsoft/Phi-3-medium-4k-instruct"
model = outlines.models.transformers(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
with open("examples.json",'r') as fin:
    complaint_data = json.loads(fin.read())

In [4]:
from pydantic import BaseModel, Field, constr
from enum import Enum


class Department(str, Enum):
    clothing = "clothing"
    electronics = "electronics"
    kitchen = "kitchen"
    automotive = "automotive"

class ComplaintData(BaseModel):
    first_name: str
    last_name: str
    order_number: str = Field(pattern=r'[ADZ][0-9]{2}-[0-9]{4}')
    department: Department

In [22]:
def create_prompt(complaint):
    complaint_messages = [
        {
        'role': 'user',
        'content': f"""
        You are a complaint processing assistent, you aim is to process complaints and return the following intformation in this JSON format:
        {{
            'first_name': <first name>,
            'last_name': <last name>,
            'order number': <order number has the following format (ADZ)XX-XXXXX>,
            'department': <{"|".join([e.value for e in Department])}>,
        }}
        """},
        {'role': 'assistant',
         'content': "I undersand and will process the complaints in the JSON format you described"
        },
        {'role': 'user',
        'content': complaint['message']
        }
    ]
    complaint_prompt = tokenizer.apply_chat_template(complaint_messages, tokenize=False)
    return complaint_prompt

In [23]:
complaint_processor = outlines.generate.json(model, ComplaintData)

In [24]:
results = []
for complaint in complaint_data:
    prompt = create_prompt(complaint)
    result = complaint_processor(prompt)
    results.append(result)

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
You are not running the flash-attention implementation, expect numerical differences.


In [38]:
idx = 3
print(f"{results[idx].order_number} vs {complaint_data[idx]['order_number']}")

A34-8956 vs My order was A348956
